In [1]:
import re
import math
import sys

import numpy
import pandas

import ipywidgets as widgets
from IPython.display import display, clear_output

import gensim
from gensim.corpora import Dictionary
from gensim.models import TfidfModel

import pythainlp

from sklearn.cluster import KMeans

from data_tokenizer import load_corpus

### Load Data

In [2]:
file_name = 'ผู้บริโภค - TescoLotus.txt'

corpus, labels = load_corpus('../data/facebook/' + file_name)

len_corpus = len(corpus)
print('Total documents', len_corpus)

clusters = list(set(labels))
print(len(clusters), 'clusters')

f = open('../data/facebook/tokenized/tokenized_' + file_name)
tokenized_corpus = eval(f.read())
f.close()

Total documents 269
1 clusters


### Preprocess Corpus

#### Remove Words

In [26]:
dictionary = Dictionary(tokenized_corpus)
print('origin:', len(dictionary), 'words')

dictionary.filter_extremes(no_below=2, no_above=0.7, keep_n=len(dictionary))
print('filter frequent words:', len(dictionary), 'words')

letter_words = [id for id in range(len(dictionary)) if len(dictionary[id]) <= 1] 
dictionary.filter_tokens(bad_ids=letter_words)
print('filter letter words:', len(dictionary), 'words')

stopwords = pythainlp.corpus.stopwords.words('thai')
stopwords.append('นี้')
dictionary.add_documents([stopwords])
stopwords = [dictionary.token2id[word] for word in stopwords]
dictionary.filter_tokens(bad_ids=stopwords)
print('filter stop words:', len(dictionary), 'words')

origin: 1313 words
filter frequent words: 540 words
filter letter words: 539 words
filter stop words: 352 words


In [24]:
# bow_corpus = [dictionary.doc2bow(doc) for doc in tokenized_corpus]
idx_corpus = [dictionary.doc2idx(doc) for doc in tokenized_corpus]

temp_corpus = []
for doc in idx_corpus:
    temp_corpus.append([dictionary[id] for id in doc if id >= 0])
idx_corpus = temp_corpus

#### Dimension Reduction

In [30]:
average_doc_size = 0
for doc in idx_corpus:
    average_doc_size += len(doc)
average_doc_size /= len(idx_corpus)
average_doc_size = math.ceil(average_doc_size)
average_doc_size

df = dictionary.dfs
filtered_corpus = []
for doc in idx_corpus:
    new_doc = [(word, df[dictionary.token2id[word]]) for word in doc]
    new_doc.sort(reverse=True, key=lambda x: x[1])
    new_doc = new_doc[:average_doc_size]
    filtered_corpus.append([word for word, df in new_doc])

### K-Mean

In [6]:
def get_bow(corpus):
    new_dict = Dictionary(corpus)

    # new_dict.filter_extremes(no_below=2, no_above=1, keep_n=len(new_dict))
    # print(len(new_dict))

    unique_words = [new_dict[id] for id in range(len(new_dict))]
    array = numpy.zeros((len_corpus, len(unique_words)), dtype=float)
    
    for i, doc in enumerate(corpus):
        for word in doc:
            array[i, new_dict.token2id[word]] += 1

        ## normalization
        if len(doc) != 0:
            array[i] = numpy.divide(array[i], len(doc))

    return pandas.DataFrame(array, columns=unique_words, dtype=float)

In [7]:
def predict_cluster(bow_corpus, num_clusters):
    kmeans = KMeans(n_clusters=num_clusters).fit(bow_corpus)
    predicted_labels = kmeans.labels_

    result = pandas.DataFrame()
    result['comment'] = corpus
    result['tokenized_comment'] = idx_corpus
    result['label'] = labels
    result['predicted_label'] = predicted_labels
    
    return result

In [35]:
num_clusters = 6
# bow_corpus = get_bow(idx_corpus)
bow_corpus = get_bow(filtered_corpus)
result = predict_cluster(bow_corpus, num_clusters)
result.head()

,comment,tokenized_comment,label,predicted_label
0,บางครั้งพนักงานโลตัสเอ็กเพลส กับพนักงานโลตัสให...,"[พนักงาน, พนักงาน, โลตัส, โลตัส, ทำ, เอ็กเพลส,...",,0
1,เลิกเปิดเพลง ข้าวแสนดี กับอีเครื่องกรองน้ำเพีย...,"[เลิก, ข้าว, ดี, อี, เครื่อง, น้ำ]",,5
2,ถุงงง ถุงมึงบอบบางมากกก ยิ่งเจอน้ำยาซักผ้า น้...,"[ถุง, งง, ถุง, กก, เจอ, น้ำยา, ซัก, ผ้า, น้ำ, ...",,2
3,เราไม่เคยรับถุงเลย แถมได้แต้มกรีนพ้อยไว้ใช้เป็...,"[ถุง, แถม, ลด, ลอง, ดู, ถุง, หนา]",,2
4,เคยไปทำ เอ้กซ์เพรส เดือนเดียว พอเลย พนักงานต่...,"[ทำ, เดือน, พนักงาน, คน, คน, คน, นึง, พัน, เช้...",,5


### Result

In [9]:
label_count = numpy.unique(result['predicted_label'], return_counts=True) 
print(label_count)

for cluster in clusters:
    print('\t' + cluster, end='')
print('\tpercent')

for label in range(len(clusters)):
    print(str(label) + '  |', end='')
    
    num_max = 0
    for cluster in clusters:
        loc = result[(result['label'] == cluster) & (result['predicted_label'] == label)]
        if len(loc) > num_max:
            num_max = len(loc)
        print('\t' + str(len(loc)), end='')
    
    print('\t' + str(num_max / label_count[1][label]))

(array([0, 1, 2, 3, 4, 5], dtype=int32), array([  4, 100, 133,  10,  13,   9]))
		percent
0  |	4	1.0


In [20]:
comment_widget = widgets.ToggleButtons(
    options=[num + 1 for num in range(num_clusters)],
    disabled=False,
    button_style='',
)

def on_comment_widget_click(change):
    clear_output()
    display(comment_widget)
    for index, value in result[result['predicted_label'] == change['new']]['comment'].iteritems():
        print(index, value)

comment_widget.observe(on_comment_widget_click, names='index')
on_comment_widget_click({'new' : 0})

ToggleButtons(index=4, options=(1, 2, 3, 4, 5, 6), value=5)

0 บางครั้งพนักงานโลตัสเอ็กเพลส กับพนักงานโลตัสใหญ่ ก็ต่างกันครับ โลตัสใหญ่ เขาทำเป็นเเผนกๆไปครับ เเต่โลตัสเอ็กเพลสนั้น พนักงานเเต่ละคน ต้องทำทุกอย่างครับ ไม่ใช่เเค่ยืนขายของ อยากให้ทุกคนเข้าใจด้วยครับ บางคนเวลากินข้าวเเทบไม่มี ผมเชื่อว่าทุกคนก็เป็นพนักงานเงินเดือนเหมือนกัน ควรเข้าใจกันบ้างครับ พนักงานคนอื่นเป็นเเบบไหนผมไม่รู้ เเต่มันก็ยังมีพนักงานที่ตั้งใจทำงานครับ ขอให้ทุกคนเข้าใจครับ
39 90 เปอร์เซ็นต์ของคอมเม้นต์นี่ เกี่ยวกับพนักงานล้วนๆ 555รวมทั้งของนี่ด้วย
116 พนักงานขายที่นี่มีการคัดกรองมั้ยคะคุณสมบัติพนักงานที่นี่
184 พนักงานตามเอ็กเพลสนี่ต้องฝึกมารยาทนะ เห็นอยู่กันตามยถากรรม
185 มารยาทพนักงาน
188 ปรับปรุงพนักงานค่ะ
221 พนักงาน
228 มารยาทของพนักงาน
233 เอ๊กเพรส ต้องปรับปรุงพนักงาน
236 พนักงาน
238 พนักงาน
241 มารยาทพนักงานควรปรับปรุงมากที่สุด
253 มารยาทของพนักงานของพนักงานค่ะ


In [16]:
token_widget = widgets.ToggleButtons(
    options=[num + 1 for num in range(num_clusters)],
    disabled=False,
    button_style='',
)

def on_token_widget_click(change):
    clear_output()
    display(token_widget)
    for index, value in result[result['predicted_label'] == change['new']]['tokenized_comment'].iteritems():
        print(index, value)

token_widget.observe(on_token_widget_click, names='index')
on_token_widget_click({'new' : 0})

ToggleButtons(index=4, options=(1, 2, 3, 4, 5, 6), value=5)

0 ['พนักงาน', 'พนักงาน', 'โลตัส', 'โลตัส', 'ทำ', 'เอ็กเพลส', 'พนักงาน', 'คน', 'ทำ', 'ขาย', 'คน', 'คน', 'เวลา', 'กิน', 'ข้าว', 'ผม', 'คน', 'พนักงาน', 'เหมือน', 'พนักงาน', 'คน', 'ผม', 'รู้', 'พนักงาน', 'ตั้งใจ', 'ทำ', 'งาน', 'คน']
39 ['พนักงาน']
116 ['พนักงาน', 'ขาย', 'พนักงาน']
184 ['พนักงาน', 'มารยาท']
185 ['มารยาท', 'พนักงาน']
188 ['ปรับปรุง', 'พนักงาน']
221 ['พนักงาน']
228 ['มารยาท', 'พนักงาน']
233 ['ปรับปรุง', 'พนักงาน']
236 ['พนักงาน']
238 ['พนักงาน']
241 ['มารยาท', 'พนักงาน', 'ปรับปรุง']
253 ['มารยาท', 'พนักงาน', 'พนักงาน']
